In [ ]:
bronze_path = 'abfss://nytaxy@dlsunaihbd4.dfs.core.windows.net/bronze/'
silver_path = ''

In [ ]:

from pyspark.sql.types import *

table_name = 'ny_taxis'

df = spark.read.format("parquet").load(bronze_path)

df.write.mode("overwrite").format("delta").option("path", silver_path + table_name).saveAsTable("silver_" + table_name)

# show 5 rows from the table, only showing the following columns: vendorID, passengerCount, tripDistance, totalAmount
spark.sql("SELECT vendorID, passengerCount, tripDistance, totalAmount FROM silver_" + table_name).show(5)

df_ny_taxis_2019=spark.read.table("default.silver_ny_taxis_2019")

#Lets discard some data that wont be used, during our anlysis: remove following columns: doLocationId, puLocationId, storeAndFwdFlag and vendorID
df_ny_taxis_2019 = df_ny_taxis_2019.drop("doLocationId", "puLocationId", "storeAndFwdFlag", "vendorID")

# show 5 rows from df_ny_taxis_2019, only showing the following columns: passengerCount, tripDistance, totalAmount
df_ny_taxis_2019.select("passengerCount", "tripDistance", "totalAmount").show(5)

#Lets measure the Tip percentage, based on the columns totalAmount and tipAmount. Create a column called tipPercentage.
df_ny_taxis_2019 = df_ny_taxis_2019.withColumn("tipPercentage", (df_ny_taxis_2019.tipAmount / df_ny_taxis_2019.totalAmount) * 100)

#Measure the price per mile for every service offered, based on the colums totalAmount and tripDistance. Create a column called "pricePerMile"
df_ny_taxis_2019 = df_ny_taxis_2019.withColumn("pricePerMile", df_ny_taxis_2019.totalAmount / df_ny_taxis_2019.tripDistance)

#aggregation for most frequent payment type and average tip percentage (name column avgTipPercentage)
df_agg2=df_ny_taxis_2019.groupBy("paymentType").agg({"tipPercentage": "avg"})
#rename column
df_agg2=df_agg2.withColumnRenamed("avg(tipPercentage)", "avgTipPercentage")








